## Prototype of the bundling interface

We start by importing the `BundleCoordinator` class, which will allow to assign the atomic maps to different bundles.

In [1]:
import sys
sys.path.insert(1, "../../pipeline/bundling")
from coordinator import BundleCoordinator

We then specify the path to the atomic maps database, input for the bundling.

In [2]:
atomic_db = "/pscratch/sd/s/susannaz/shared/SO_ISO/satp3_maps/cmb_map_satp3_8May2024/atomic_maps.db"

Time to create bundles ! We will do it for `n_bundles = 2`, a fixed seed and tell the `BundleCoordinator` to also assign maps to `high` and `low` splits for the properties listed in `null_props`.

In [3]:
bundles = BundleCoordinator(atomic_db, n_bundles=2, seed=1234, null_props=["pwv", "elevation"])

Once done, we can access to the median that was computed for the property `pwv`

In [4]:
bundles.null_props_stats["pwv"]

0.0

We then save the database to disk, so that we can release it with the bundled maps.

In [5]:
bundles.save_db("bundles_car.db")

Now, if we need to get info about bundles in a separate script, we can load it from file by calling the following method. Note that you can only load part of the database by specifying the variables `bundle_id` and `null_prop_val` (which could be `high_pwv` for example)

In [6]:
bundle_load = BundleCoordinator.from_dbfile(
    "bundles_car.db",
    bundle_id=0,
    null_prop_val=None
)

We now have all the ingredients to actually coadd the atomic maps. Before this, we never had to open any atomic map file to generate the database. For this, we import a new class : `Bundler`.

In [7]:
from coadder import Bundler

This class needs the paths to the atomic database and bundling database. There, `obs_id` is not enough to load map files, we will add further informations about the frequency and the wafer to use. Note that setting the `wafer` argument to `None` will just load and coadd all of them.
Once the bundler object instantiated, one can run the `bundle` method to coadd maps. For this you will have to specify which bundle you want to output, as well as which null property (if `None` it will just produce the science bundle)

In [8]:
bundler = Bundler(atomic_db, "bundles_car.db", freq_channel="f150", wafer="ws0",
                  pix_type="car")
map, hits = bundler.bundle(bundle_id=0, null_prop_val="low_pwv")

dec_cut (-75, 25)


Now we can visualize the map with `pixell.enplot`.

In [9]:
from pixell import enplot
enplot.plot(map[1]*1e6, colorbar=True, ticks=10, min=-100, max=100)

[Bunch(base='', ci=0, cn=1, comp='', dir='', ext='png', fcomp='', fi=0, fn=1, iext='', img=<PIL.Image.Image image mode=RGBA size=2218x657 at 0x7FC9A46D64D0>, info=Bunch(bounds=array([[ -29,  -19],
        [2189,  638]]), names=['img', 'grid', 'tics', 'colorbar']), layer='', name='.png', pi=0, pn=1, pre='', printer=<pixell.enplot.Printer object at 0x7fc9a6d1b010>, suf='', type='pil')]

To generate noise simulation, one can import and instantiate an object of the `SignFlip` class. Since we might want to generate multiple sign-flip noise realizations, we instantiate the object specifying also `bundle_id` and the null property value `null_prop_val`.

In [ ]:
from coadder import SignFlip
signflip = SignFlip(
    atomic_db, "bundles_car.db", freq_channel="f150", wafer=None, bundle_id=0,
    null_prop_val=None, pix_type="car"
)

OperationalError: no such column: median_weight_qu

In [ ]:
noise = signflip.signflip(seed=1234)
noise2 = signflip.signflip(seed=1235)
noise3 = signflip.signflip(seed=1236)

In [ ]:
enplot.plot(noise[1]*1e6, colorbar=True, ticks=10, min=-100, max=100)
enplot.plot(noise2[1]*1e6, colorbar=True, ticks=10, min=-100, max=100)
enplot.plot(noise3[1]*1e6, colorbar=True, ticks=10, min=-100, max=100)